In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = pd.read_csv('RAW_recipes.csv')

In [12]:
df2=pd.read_csv('RAW_interactions.csv')

In [13]:
df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8.0


In [14]:
df2.head()
df2=df2.drop(['user_id','date','review'],axis=1)

In [15]:
df2.head()

,recipe_id,rating
0,40893.0,4.0
1,40893.0,5.0
2,44394.0,4.0
3,85009.0,5.0
4,85009.0,5.0


In [17]:

C=df2['rating'].mean()

In [18]:
df3=df2.groupby(['recipe_id'])['rating'].agg(['mean','count'])
df3.reset_index(inplace=True)

In [19]:

df3.rename(columns={'mean':'average_rating','count':'votes'},inplace=True)

In [20]:
#Merge ratings dataframe with recipe Dataframe
food_df = pd.merge(df, df3, left_on='id',right_on='recipe_id')

In [21]:
#Show the number of votes possesed by the top 10% meals, based on number of votes
q=df3['votes'].quantile(0.9)
print(q)

8.0


In [22]:
#Get meals which have received a substantial number of votes
top_meals=food_df.copy().loc[df3['votes']>=q]

In [23]:
#This function permits us to compute a special metric which takes into account the number of people
#that have voted for a meal and not just it's average rating
def weighted_rating(x, m=q, c=C):
    v = x['votes']
    R = x['average_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [24]:
#Select top meals from Dataframe
top_meals['Score']=top_meals.apply(weighted_rating, axis=1)
food_df['Score']=food_df.apply(weighted_rating, axis=1)

In [25]:
#Sort best meals based on their scores
top_meals=top_meals.sort_values('Score',ascending=False)

#Print the top 20 meals
top_meals[['name', 'votes', 'average_rating', 'Score']].head(20)

,name,votes,average_rating,Score
0,sangria fruit cups non alcoholic,1,5.0,4.527546


In [26]:
food_df.columns

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients', 'recipe_id', 'average_rating', 'votes', 'Score'],
      dtype='object')

In [27]:
food_df.loc[0]['nutrition']

'[122.1, 0.0, 104.0, 4.0, 4.0, 0.0, 9.0]'

In [28]:
#We import literal eval to convert objects into corresponding python objects
from ast import literal_eval

In [29]:
features=['steps','ingredients']
for feature in features:
    food_df[feature] = food_df[feature].apply(literal_eval)